In [ ]:
!unzip "/content/drive/MyDrive/DermatoAI/SetDateClasificareBinaraPiele.zip" -d "/content/drive/MyDrive/DermatoAI"

In [ ]:
import os
import csv

# Path to your CSV file
csv_file_path = '/content/sample_data/Dataset-agumentat-mediu-undersampling/metadata.csv'

# Directory where your images are stored
image_directory = '/content/sample_data/Dataset-agumentat-mediu-undersampling/dataset'

missing_files = []
i = 0
# Assuming your CSV has headers and 'isic_id' is the column with image filenames
with open(csv_file_path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        isic_id = row['isic_id']  # Adjust the key if your column name is different
        image_path = os.path.join(image_directory, f"{isic_id}")  # Adjust extension if needed
        if not os.path.exists(image_path):
            missing_files.append(isic_id)

# Print the number of missing files and some examples
print(f"Total missing files: {len(missing_files)}")
if len(missing_files) > 0:
    print("Examples of missing files:", missing_files[:10])



In [3]:
!cp -r "/content/drive/MyDrive/DermatoAI/SetDateClasificareBinaraPiele" "/content/sample_data"

In [15]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Flatten, GlobalAveragePooling2D, Dropout, Conv2D, MaxPooling2D, Dense, BatchNormalization
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import numpy as np
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + K.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

np.random.seed(42)
target_size = (450, 600)
batch = 16
epochs = 10

image_directory = '/content/sample_data/SetDateClasificareBinaraPiele'

datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.3)  # 20% data for validation

train_generator = datagen.flow_from_directory(
    directory=image_directory,
    target_size=target_size,
    batch_size=batch,
    shuffle=True,
    class_mode='binary',  # Change to 'binary' for binary classification
    subset='training',
    seed=42
)

validation_generator = datagen.flow_from_directory(
    directory=image_directory,
    target_size=target_size,
    batch_size=batch,
    shuffle=False,
    class_mode='binary',  # Change to 'binary' for binary classification
    subset='validation',
    seed=42
)

print(train_generator.class_indices)


model = Sequential([
    Conv2D(16, (3, 3), activation='relu', input_shape=(target_size[0], target_size[1], 3), padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(32, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(64, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),

    Dense(1, activation='sigmoid')
])

model.summary()

model.compile(
    optimizer=Adam(learning_rate=0.0007),
    loss=BinaryCrossentropy(),
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), F1Score()]
)

early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor the validation loss
    min_delta=0.001,     # Minimum change to qualify as an improvement
    patience=2,          # Stop after 5 epochs with no improvement
    mode='min',          # Stop when the quantity monitored has stopped decreasing
    verbose=1,
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity.
)

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // batch,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // batch,
    epochs=epochs,
    callbacks=[early_stopping]
)

# Save the model
model.save('/content/drive/MyDrive/DermatoAI/Models/binary_model_3.h5')


Found 4143 images belonging to 2 classes.
Found 1775 images belonging to 2 classes.
{'healthy': 0, 'unhealthy': 1}
Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_38 (Conv2D)          (None, 450, 600, 16)      448       
                                                                 
 batch_normalization_58 (Ba  (None, 450, 600, 16)      64        
 tchNormalization)                                               
                                                                 
 max_pooling2d_38 (MaxPooli  (None, 225, 300, 16)      0         
 ng2D)                                                           
                                                                 
 conv2d_39 (Conv2D)          (None, 225, 300, 32)      4640      
                                                                 
 batch_normalization_59 (Ba  (None, 225, 300, 32)      128       
 tch

In [2]:
import os

def count_files(directory):
    total_files = 0
    for root, dirs, files in os.walk(directory):
        total_files += len(files)
    return total_files
print(count_files('/content/drive/MyDrive/DermatoAI/SetDateClasificareBinaraPiele'))

5918


In [ ]:
from google.colab import drive
drive.mount('/content/drive')